<a href="https://colab.research.google.com/github/GulnazaS/Q-A_BERT/blob/main/BERT_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка и импорт библиотек

In [ ]:
!pip install datasets
!pip install weave
!pip install streamlit
!pip install pyngrok

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DefaultDataCollator, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch
from pyngrok import ngrok

# Загрузка датасета и его предобработка


In [ ]:
squad = load_dataset("squad")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
data_collator = DefaultDataCollator()

# Fine-tuning


In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!wandb login

wandb: Currently logged in as: gulnazsahapova1998 (gulnazsahapova1998-t) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.200600,1.161279
2,0.968000,1.098293
3,0.738200,1.153556


TrainOutput(global_step=16425, training_loss=1.0819929399882278, metrics={'train_runtime': 11982.3522, 'train_samples_per_second': 21.932, 'train_steps_per_second': 1.371, 'total_flos': 2.575143204683213e+16, 'train_loss': 1.0819929399882278, 'epoch': 3.0})

In [ ]:
torch.save(model.state_dict(), 'bert_fine_tune.pth')

In [ ]:
# Очистка кеша памяти GPU
torch.cuda.empty_cache()

# Deploy on Streamlit

In [3]:
%%writefile app.py
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

st.title("BERT к Вашим услугам")

# Функция для загрузки модели
@st.cache_resource  # Кэшируем модель, чтобы не загружать её каждый раз
def load_model():
    # Укажите путь к вашей предобученной модели
    model_path = "/content/bert_fine_tune.pth"
    # Загрузите модель
    model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))  # Исправлено: torch.device
    model.eval()
    return model

# Загрузите токенизатор
@st.cache_resource  # Кэшируем токенизатор
def load_tokenizer():
    return AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Загрузите модель и токенизатор
model = load_model()
tokenizer = load_tokenizer()

# Инициализация истории чата
if "messages" not in st.session_state:
    st.session_state.messages = []

# Отображение сообщений из истории чата
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Функция для предсказания
def predict(question, context):
    # Токенизация входных данных (вопрос + контекст)
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Предсказание
    with torch.no_grad():
        outputs = model(**inputs)

    # Получение start_logits и end_logits
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Нахождение индексов начала и конца ответа
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    # Преобразование индексов в текст
    answer_tokens = inputs["input_ids"][0][start_index:end_index + 1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    return answer

# Интерфейс Streamlit
st.write("Введите контекст и вопрос, чтобы получить ответ.")

# Поле для ввода контекста
context = st.text_area("Введите контекст:")

# Поле для ввода вопроса
question = st.text_input("Введите Ваш вопрос:")

if st.button("Ответить на вопрос"):
    if question and context:  # Проверка наличия вопроса и контекста
        # Получаем предсказание
        answer = predict(question, context)
        # Выводим результат
        st.write(f"Ответ: {answer}")

        # Добавляем вопрос и ответ в историю чата
        st.session_state.messages.append({"role": "user", "content": question})
        st.session_state.messages.append({"role": "assistant", "content": answer})
    else:
        st.write("Пожалуйста, введите вопрос и контекст.")

Overwriting app.py


In [4]:
!nohup streamlit run app.py &>/dev/null &

In [ ]:
ngrok_token = input("Введите ваш ngrok Authtoken: ")
ngrok.set_auth_token(ngrok_token)

# Запустите ngrok для порта 8501 (Streamlit)
public_url = ngrok.connect(addr='8501', proto='http')
print("Streamlit доступен по адресу:", public_url)